In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV fil/kaggle/input/se-text-classifier-hackerrank/input00.txte I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/se-text-classifier-hackerrank/input00.txt
/kaggle/input/training-se-text-classifier-hackerrank/training.json


In [18]:

import codecs
import json 
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier 

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split 

In [19]:
with codecs.open('/kaggle/input/training-se-text-classifier-hackerrank/training.json', 'r', encoding='iso-8859-1') as f:
    data = f.read()
    
data = data.split('\n')

In [8]:
# data

In [20]:
N = int(data[0]) 
question = []
excerpt = []
topic = []
row = []

for i in range(1, N+1): 
    y = json.loads(data[i])
    question.append(y['question'])
    topic.append(y['topic'])
    excerpt.append(y['excerpt'])
    
    
    rows = [y['topic'], y['question'], y['excerpt']]
    
    row.append(rows)

colnames= ['topic', 'question', 'excerpt']
training_df = pd.DataFrame(row, columns = colnames)


# for i in range(1, N+1): 
#     y = json.loads(data[i])
#     question.append(y['question'])
#     topic.append(y['topic'])
#     excerpt.append(y['excerpt'])
    
    
#     rows = [y['topic'], y['question'], y['excerpt']]
    
#     row.append(rows)

# colnames= ['topic', 'question', 'excerpt']
testing_df = pd.DataFrame(row, columns = colnames)

In [21]:
training_df

,topic,question,excerpt
0,electronics,What is the effective differencial effective o...,"I'm trying to work out, in general terms, the ..."
1,electronics,Heat sensor with fan cooling,Can I know which component senses heat or acts...
2,electronics,Outlet Installation--more wires than my new ou...,I am replacing a wall outlet with a Cooper Wir...
3,electronics,Buck Converter Operation Question,"i have been reading about the buck converter, ..."
4,electronics,"Urgent help in area of ASIC design, verificati...",I need help with deciding on a Master's Projec...
...,...,...,...
20214,wordpress,How to set a Custom Post Type as the parent of...,I have a Custom Post Type called Recipe with p...
20215,wordpress,Tracking last login and last visit,I'm using the code below to track when a user ...
20216,wordpress,How to exclude the particular category from th...,"add_action( 'pre_get_posts', 'custom_pre_get_p..."
20217,wordpress,display sub categories assoccited with each po...,i have wordpress blog with many posts. each po...


In [22]:
def data_inspection(df): 
    
    missing_values = df.isnull().sum()
    percent_missing_values = df.isnull().sum()/df.apply(len)
    
    print(missing_values)
    print(percent_missing_values)
    
    label_counts = df['topic'].value_counts()
    percent_label_counts = df['topic'].value_counts(normalize= True)*100
    
    print(label_counts)
    print(percent_label_counts)
    
    
    predictor = 'question'
    df['doc_length'] = df[predictor].apply(len)
    
    min_doc_length = df[predictor].apply(len).min()
    max_doc_length = df[predictor].apply(len).max()
    mean_doc_length = df[predictor].apply(len).mean()
    median_doc_length = df[predictor].apply(len).median()
    first_quartile_doc_length = df[predictor].apply(len).quantile(0.25)
    third_quartile_doc_length = df[predictor].apply(len).quantile(0.75)
    
    print(min_doc_length)
    print(max_doc_length)
    print(mean_doc_length)
    print(median_doc_length)
    print(first_quartile_doc_length)
    print(third_quartile_doc_length)
    
data_inspection(training_df)
# data_inspection(testing_df)

topic       0
question    0
excerpt     0
dtype: int64
topic       0.0
question    0.0
excerpt     0.0
dtype: float64
gis            2383
scifi          2333
android        2239
electronics    2079
apple          2064
unix           1965
photo          1945
wordpress      1943
security       1899
mathematica    1369
Name: topic, dtype: int64
gis            11.785944
scifi          11.538652
android        11.073743
electronics    10.282408
apple          10.208220
unix            9.718582
photo           9.619665
wordpress       9.609773
security        9.392156
mathematica     6.770859
Name: topic, dtype: float64
15
157
54.134032345813345
51.0
39.0
66.0


In [23]:
vectorizer = TfidfVectorizer(lowercase= True, stop_words = 'english')


# model = MultinomialNB() dicrete feature text classification 
model = RandomForestClassifier(random_state=100)

X = vectorizer.fit_transform(training_df['question'] + '' + training_df['excerpt'])
y = training_df['topic']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=100)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred, digits=4)

print (report)


              precision    recall  f1-score   support

     android     0.8736    0.8897    0.8816       544
       apple     0.8710    0.7383    0.7992       512
 electronics     0.8655    0.8178    0.8410       527
         gis     0.8987    0.8712    0.8847       621
 mathematica     0.8071    0.8030    0.8051       396
       photo     0.8873    0.9174    0.9021       472
       scifi     0.7765    0.9421    0.8513       553
    security     0.8473    0.7100    0.7726       469
        unix     0.7206    0.7664    0.7428       488
   wordpress     0.8566    0.9091    0.8821       473

    accuracy                         0.8394      5055
   macro avg     0.8404    0.8365    0.8362      5055
weighted avg     0.8422    0.8394    0.8385      5055



In [28]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[484   7   8   4   3   8  12   5   9   4]
 [ 26 378   3   2   5  15  18  17  39   9]
 [  7   2 431   5  16  10  35   8   8   5]
 [  6   4   8 541  14   2  11   5  21   9]
 [  1   3  10  17 318   4  17   1  17   8]
 [  5   4   7   6   0 433  13   2   2   0]
 [  3   0   6   3   6  10 521   1   0   3]
 [ 14  17   7   8   8   1  19 333  41  21]
 [  5  17  16  10  12   2  20  19 374  13]
 [  3   2   2   6  12   3   5   2   8 430]]


In [34]:
result_df = pd.DataFrame(y_eval, columns=['pred_topic'])
result_df


,pred_topic
0,electronics
1,electronics
2,electronics
3,electronics
4,electronics
...,...
20214,wordpress
20215,wordpress
20216,wordpress
20217,wordpress


In [36]:
# path = r'/kaggle/working/result_topic.txt'

# with open(path, 'a') as f: 
#     df_string = result_df.to_string(index = False, header = False )
#     f.write(df_string)
    
result_df.to_csv(r'/kaggle/working/result_topic.txt', mode='a', index=False, header=False)